## Pipeline Script 
This script 

### Import Packages

In [6]:
import os
import glob
import numpy as np
import pandas as pd
#import network_fcon as fc
from scipy.stats import pearsonr

### Define paths and variables

In [65]:
# Main folder path
# subjs = "subject_list.csv"
inpath = "/Users/pecsok/Desktop/Maggie/Graduate School/Project1_GluCEST_rsfMRI/xcp_output/xcp_d_test/" 
outpath = "~/Desktop/ImageData/PMACS_remote/analysis/postprocessing"
# Choose what to analyse
networks = ["SalVentAttn", "Default"] 
CNB_scores = ["matrix", "verbal"]
clinical_scores = ["BDI", "PROMIS-Anx"]
# Make dataframe based on metrics of interest
columns = ["BBLID"] + ["Session"] + networks_of_interest
fcon_df = pd.DataFrame(columns=columns)
clin_df = pd.DataFrame(columns=["BBLID", "Session"] + CNB_scores_of_interest + clinical_scores_of_interest)
# Initialize empty lists and vars
bblids = []
sesids = []

# Initialize empty matrix for fcmats # FIX if desired:
num_matrices = 100  # You can change this to the desired number
matrix_shape = (700, 701)
index = pd.MultiIndex.from_product([range(num_matrices), range(matrix_shape[0]), range(matrix_shape[1])], names=['Matrix', 'Row', 'Column'])
# Create an empty NumPy array with NaN values to represent the data
data = np.empty((num_matrices * matrix_shape[0] * matrix_shape[1],))
data[:] = np.nan
# Create a Pandas DataFrame with the MultiIndex
fcmats = pd.DataFrame(data, index=index, columns=['Value'])

                   Value
Matrix Row Column       
0      0   0         NaN
           1         NaN
           2         NaN
           3         NaN
           4         NaN
...                  ...
99     699 696       NaN
           697       NaN
           698       NaN
           699       NaN
           700       NaN

[49070000 rows x 1 columns]


### Choose which modules to run

In [42]:
runfcon = True
runALFF = True
runCNB = True
runCEST = True
runClinical = True

### Import data, loop through subjects, and establish file paths

In [72]:
# Generate list of subjects glob.glob(inpath + 'sub-*) # Generates list of all file names
folder_names = [folder for folder in glob.glob(os.path.join(inpath, "*")) if os.path.isdir(folder)]

subjlist_md = []
allsubj_rsns = []

for subj_path in folder_names:
    # Extract bblid id:
    bblid = subj_path.split('-')[1]
    print("Processing subject " + bblid)
    # Extract session id: 
    items = os.listdir(subj_path)
    ses_folder = [item for item in items if item.startswith("ses")]
    ses = ses_folder[0].split('-')[1]
    ses_path = os.path.join(input_path, subj_path, ses_folder[0]) # full path to session
    # Add to running list of bblids for grp analysis later:
    bblids.append(bblid)
    sesids.append(ses)
    
    if runfcon:
        #subj_fcon(ses_path, subj, ses, fcmats, grp_df, networks_of_interest):
        fcmat_glob = f"{ses_path}/func/*Schaefer717_measure-pearsoncorrelation_conmat.tsv"
        fcmat = pd.read_csv(glob.glob(fcmat_glob)[0], sep='\t') # read in fcmat
        # Add to 3D matrix for group analysis later
        #fcmats = df
        #fcmats = pd.concat([fcmats, fcmat], level='Matrix')
        #fcmats.shape
        # Make list for this subject's avg network values
        avg_fc = []
        # Loop through the networks
        for network in networks:
            # Select rows and columns corresponding to the network
            #network_indices = [index for index in fcmat.index if network in index]
            print(fcmat)
            network_fc = fcmat.loc[fcmat.index.str.contains(network), fcmat.columns.str.contains(network)]
            #network_fc = fcmat.loc[network_indices, network_indices]
            # Calculate the average connectivity within the network
            avg_fc.append(network_fc.values.mean())
            print(avg_fc)
            # Calculate avg within-network connectivity for the current subject
            #avg_fcon = subj_fcon(fcmat, networks_of_interest) 
            temp_df = pd.DataFrame(bblid, columns=["BBLID"])
            temp_df["Session"] = ses
            temp_df[networks_of_interest] = avg_fcon
            # Append the subject's data as a new row to the DataFrame
            grp_df = grp_df.append(temp_df, ignore_index=True)
        
        



Processing subject 116019copy


TypeError: 'DataFrame' object is not callable

### Network FCON 
Call fcon function and calculate network-level connectivity measures for subject and add  subject data to group file for future group analysis

In [ ]:
        path_to_fcmat = f"{subj_path}/func/*Schaefer717_measure-pearsoncorrelation_conmat.tsv"
              fcmat = pd.read_csv(path_to_fcmat, sep='\t')
              # Add to 3D matrix for group analysis later
              fcmats = pd.concat([fcmats, fcmat], axis=2)
              # Calculate avg within-network connectivity for the current subject
              avg_fcon = subj_fcon(fcmat, networks_of_interest) 
              temp_df = pd.DataFrame(bblid, columns=["BBLID"])
              temp_df["Session"] = ses
              temp_df[networks_of_interest] = avg_fcon
              # Append the subject's data as a new row to the DataFrame
              grp_df = grp_df.append(temp_df, ignore_index=True)
                    
          
          
          if runalff:
              path_to_fcmat = f"{subj_path}/func/*Schaefer717_measure-pearsoncorrelation_conmat.tsv"
              fcmat = pd.read_csv(path_to_fcmat, sep='\t')
              # Add to 3D matrix for group analysis later
              fcmats = pd.concat([fcmats, fcmat], axis=2)
              # Calculate avg within-network connectivity for the current subject
              avg_fcon = subj_fcon(fcmat, networks_of_interest) 
              temp_df = pd.DataFrame(bblid, columns=["BBLID"])
              temp_df["Session"] = ses
              temp_df[networks_of_interest] = avg_fcon
              # Append the subject's data as a new row to the DataFrame
              grp_df = grp_df.append(temp_df, ignore_index=True)      
          
          if runcnb:
              
              